Relationships between Records
=========================

Until now, we've been working with `Record`s. While these are the heart and soul of Sina, they're not the only thing it can track! `Relationship`s, as you might guess, are used to tie Records together.

Let's say that you had an `msub` submit a batch of runs, but later realized there was something wrong with how it was launched. As part of your workflow, you added relationships between the `msub` and each of its launched runs, meaning you can now find all of those runs knowing only the msub that launched them. But how did you know there was something wrong with the `msub`? Well, you got back a run that looked a little strange, suspected something was up, and looked through *its* relationships and found this msub, then saw its settings looked a little off. In this way, Relationships make Records "discoverable" from one another. You could check a run's msub's data, you could check all the runs submitted by an msub. Relationships are like paths, bridging together different pieces of your workflow.


`Relationship`s are extremely simple, lightweight, and are meant to be read almost like sentences. `msub_1` `submits` `record_1` is a Relationship, so is `record_462_edit` `corrects` `record_462`. They look just like that in the `datastore`, and they're used just like that in the code.

As simple as they are, though, the flexibility of `Relationship`s is what allows Sina to represent entire workflows, connect parent metadata with children, and protect data pedigree. Let's make a few!

Making a few basic Records
---------------------------------

Nothing new here. We'll create three Records, one representing an msub, two representing simulation runs submitted by it, and add them to a datastore.

`Relationship`s can only be created between existing `Record`s! If one (or both) of its `Record`s are deleted, the orphaned `Relationship` is automatically deleted, too. Hence our need to create some Records first.

In [ ]:
import sina
import sina.utils
from sina.model import Record, Relationship

msub_rec = Record(id="msub_1", type="msub")
msub_rec.add_data("machine", "quartz")
msub_rec.add_data("node", 49)

run_1_rec = Record(id="run_1", type="foo_sim_run")
run_1_rec.add_data("etot", 1983.23)

run_2_rec = Record(id="run_2", type="foo_sim_run")
run_2_rec.add_data("etot", 2092.45)

recs_to_insert = [msub_rec, run_1_rec, run_2_rec]
ds = sina.connect()
ds.records.insert(recs_to_insert)

print("Inserted: {}".format(", ".join(x.id for x in recs_to_insert)))

Adding Relationships
-------------------------

Relationships consist of a `subject_id`, `predicate`, and `object_id`, which are together read like a sentence. The only rule (besides the aforementioned need for both ids to already exist) is that the `predicate` should always be in an active voice. Use `record_462_edit` `corrects` `record_462`, NOT `record_462` `is_corrected_by` `record_462_edit`. While the latter won't "break" anything, it can be a headache trying to remember what order things go in, so using one voice throughout (preferably active) can save you trouble later.

Predicates can be *anything* which, paradoxically, can make them difficult to choose. "Submits" (msub submits run), "corrects" (correction_run corrects run), "requests" (agent requests run), "launches" (script launches ensemble), and "uses" (run uses deck) are some common choices, though no predicate has more (or less) support from Sina itself. Of course you can add new Relationships (and delete old ones), and the operation's very lightweight, should you change your mind on what predicate to use.

In [ ]:
predicate = "submits"

# Note how we specify subject_id=... !
# This is an unfortunate holdover from Sina's early days, as object_id was written as the first arg.
# While easy to correct, swapping the order to the "proper" one (subject_id first) would
# constitute an API break, and is thus being saved for a major update.
msub_rec_1_rel = Relationship(subject_id=msub_rec.id, predicate=predicate, object_id=run_1_rec.id)
msub_rec_2_rel = Relationship(subject_id=msub_rec.id, predicate=predicate, object_id=run_2_rec.id)

rels_to_insert = [msub_rec_1_rel, msub_rec_2_rel]
# ds.relationships, not ds.records!
ds.relationships.insert(rels_to_insert)
print("Inserted: {}".format(", ".join("relationship between {} & {}"
                                      .format(x.subject_id, x.object_id) for x in rels_to_insert)))

Selecting Relationships
----------------------------

With Relationships being as simple as they are, there's only one query for finding them, but it's quite flexible. You can specify `subject_id`, `predicate`, `object_id`, or any combination of the 3. You'll get back the Relationship itself, which holds all 3 fields.

In [ ]:
print("Relationships with {} as the subject_id:\n{}"
      .format(msub_rec.id,
              "\n".join(str(x) for x in ds.relationships.find(subject_id=msub_rec.id))))
print("\nRelationships where {} is the subject and {} is the object:\n{}"
      .format(msub_rec.id, run_1_rec.id,
              "\n".join(str(x) for x in ds.relationships.find(subject_id=msub_rec.id, object_id=run_1_rec.id))))
print("\nRelationships with a predicate of \"{}\":\n{}"
      .format(predicate,
              "\n".join(str(x) for x in ds.relationships.find(predicate=predicate))))

Getting Data via Relationships
------------------------------------
Finally, you can combine `Relationship` and `Record` queries to get data from related `Record`s. This is a nice way to have a single source of truth shared between your runs.

In [ ]:
run_2_msub = ds.relationships.find(predicate="submits", object_id=run_2_rec.id)[0].subject_id

print("Record {} was run on node {}"
      .format(run_2_rec.id,
              ds.records.get(run_2_msub).data_values["node"]))

Of course, you can do this in the other direction too, like getting all the `etot`s tied to an msub.